In [1]:
import pandas as pd
import numpy as np
import math

from pandas_profiling import ProfileReport

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
movies_df = pd.read_csv('movie_metadata.csv')

#creates a profile document which analyses each of the variables in the dataframe 
# movies_profile = ProfileReport(movies, title = "Movie Report", explorative=True)
# movies_profile.to_file("movies.html")

# Information Gained From the Profile

**Numerical**

director_facebook_likes, *duration*,  actor_1_facebook_likes, actor_2_facebook_likes, actor_3_facebook_likes, cast_total_facebook_likes, movie_facebook_likes, 

num_voted_users, num_critic_for_reviews,  num_user_for_reviews   

gross, budget,

*imbd_score*, facenumber_in_poster

*Italitized items have close to a normal distribution. The rest of the values are skewed.*

**Categorical**

*director_name, actor_1_name, actor_2_name, actor_3_name*

language, country, content_rating

*movie_title, movie_imdb_link*,  aspect_ratio, color

*Italitized items have high cardinality.*

**Imbalanced Variables**: color, language, country

**List Variables**: genres, plot keywords

**Large Percentage Missing**: director_facebook_likes, gross, budget, title_year

**Extreme Values** : Most of the numerical values have outliers/extreme values. 

* About half of the varaiables contained information that could be used to plan future productions. 
* The other half of the information was about things that studios would have little or no control over, such as the number of facebook likes or reviewer on imdb.
* Some of the variables, like director and actor, had a lot of different possible values.  This could make it more difficult to use with Machine Learning algorithms.
* The first movie in the database was from 1916.  Movies that are from far in the past might not be as good at helping to predict future movie sales.
* Data was heavily scewed towards color movies, english language and movies made in the US. As most movies are made in color now, this variable could probably be dropped. Language and country data would either need some processing before use or to drop them as well.
* Most of the data did not have missing values. Unfortunately, budget and gross, two of the more important ones for this analysis had a high percentage of missing values.
* Very few of the values had normal distribution.  Some of this was likely due the changes in technology over time and inflation.
* Many of the values had high correlations between them. Some of this was also likely due to changes over time.
* Both budget and gross had an extremely wide spread.  Part of this was probably due to inflation. Adjusting amounts for inflation might help with this to some extent.

In [3]:
# create a new column, transform 'genres' and 'plot_keywords' into lists, 
# store the lists in the new columns
movies_df['genre_list'] = movies_df['genres'].copy()
movies_df['genre_list'] = movies_df['genre_list'].str.split('|')

movies_df['keyword_list'] = movies_df['plot_keywords'].copy()
movies_df['keyword_list'] = movies_df['keyword_list'].str.split('|')

# create boolean columns for each genre

genres = set([item for lists in movies_df['genre_list'] for item in lists])

# Below code was taken from: 
# https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
# Create empty dict
genre_dict = {}

# Loop through all the tags
for i, item in enumerate(genres):
    # Apply boolean mask
    genre_dict[item] = movies_df['genre_list'].apply(lambda x: item in x)

# Return the results as a dataframe, change True and false values to 0 and 1
genre_frame =  pd.DataFrame(genre_dict)
genre_frame = genre_frame.astype(int)


movies_df = pd.merge(movies_df, genre_frame, left_index = True, right_index = True)        


Inflation information was found at: https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG
Some countries were not present in the data.
The countries on the left were give the values for the country or region on the right as they were closest economic match
Argentina: Latin America & Caribbean
Finland: EU
Indonesia: East Asia & Pacific (excluding high income)
South Korea: East Asia & Pacific
Taiwan: China
Thailand: East Asia & Pacific (excluding high income)
West Germany - Russia
Soviet Union - Russia

In [12]:
#load information from API
budget_data = pd.read_csv("movie_budget_data.csv")
budget_data.rename(columns = {'0': 'title', '1':'budget', '2':'gross'}, inplace = True)
# movies_df['budget'].fillna(0, inplace = True)

#replace information in database with information from API
for index, title, budget, gross in budget_data.itertuples():
    strbudget = str(budget)
    strgross = str(gross)
    strtitle = title
    if budget != 0:
        movies_df.loc[title, 'budget'] = gross
    if gross != 0:
        movies_df.loc[title, 'gross'] = gross

                          title    budget      gross
0             Napoleon Dynamite    400000   46118097
1                     Childless   1000000          0
2                Little Big Top         0          0
3                      Æon Flux  62000000   53321673
4     The Spiderwick Chronicles  90000000  164170327
...                         ...       ...        ...
1453                The Nut Job  42000000  122529966
1454     For Your Consideration  12000000    5542025
1455                       Rush  38000000   90247624
1456              Anna Karenina         0   68929150
1457     Lars and the Real Girl  12000000   11293663

[1458 rows x 3 columns]


In [16]:
inflation = pd.read_csv("price_index.csv")

# calculate mean inflation rate for each country.  Use this rate to fill in NaN values    

inflation['mean'] = inflation.mean(axis = 1, numeric_only = True)
inflation = inflation.set_index('country')

# fill missing years with 2005, missing countries with USA (most common)
movies_df['title_year'].fillna(2005, inplace = True)
movies_df['country'].fillna('USA', inplace = True)

#determine inflation rate for each movie, movies before 1960 given mean inflation rate
movies_df['title_year'] = movies_df['title_year'].astype(int)

movies_df[['budget', 'gross']].dropna()

for index, year, country in movies_df[['title_year', 'country']].itertuples():
    stryear = str(year)
    strcountry = str(country)
    if year >= 1960:
        movies_df['rate'] = inflation.loc[[strcountry],[stryear]].iloc[0][0]
    else:
        movies_df['rate'] = inflation.loc[[strcountry],['mean']].iloc[0][0]

# find budget and gross adjusted for inflation
movies_df['budget_infl'] = (movies_df['budget']/movies_df['rate']) * 100
movies_df['gross_infl'] = (movies_df['gross']/movies_df['rate']) * 100

#create column to record if movies were rated for TV and then consolidate ratings into smaller
# categories
movies_df['format'] = np.where(movies_df['content_rating'].str.match('TV'), "TV", "Movie")
movies_df['content_rating'].replace({'Not Rated': 'Unrated', 'Approved': 'Unrated', 
                                           'TV-G': 'G',  'TV-PG': 'PG', 'TV-MA': 'R',
                                           'TV-Y': 'G', 'TV-14': 'PG-13', 'Passed': 'Not Rated',
                                           'TV-Y7': 'PG', 'M': 'PG', 'GP': 'PG'}, inplace = True)
                                           
movies_df['profit'] = movies_df['gross_infl'] - movies_df['budget_infl']
movies_df.drop_duplicates(subset=['color', 'director_name', 'duration', 
                                  'actor_2_name', 'actor_1_name', 'movie_title', 'actor_3_name', 
                                  'facenumber_in_poster', 'language', 'country', 'content_rating',
                                  'budget', 'title_year', 'gross', 'aspect_ratio'], inplace = True)

In [18]:
#create a database for movies that are after 1990
movies_after_1990 = movies_df.loc[movies_df['title_year'] >= 1990]
movies_after_1990 = movies_after_1990.drop(columns = ['color', 'num_voted_users', 'num_user_for_reviews', 'num_critic_for_reviews'])


# add a column showing relative frequency of movies directed by a particular director
num_movies = movies_after_1990['director_name'].value_counts(normalize = True).reset_index()
num_movies.columns = ['director_name', 'director_count']
movies_after_1990 = pd.merge(movies_after_1990, num_movies, on = 'director_name')


# find frequency of each actor
actor1_num_movies = pd.DataFrame(movies_after_1990[['actor_1_name']].value_counts()).reset_index()
actor2_num_movies = pd.DataFrame(movies_after_1990[['actor_2_name']].value_counts()).reset_index()
actor3_num_movies = pd.DataFrame(movies_after_1990[['actor_3_name']].value_counts()).reset_index()


#create a dictionary with the total count of actors across actor1, actor2 and actor 3
actor_dict = { }

for index1, actor1, count1 in actor1_num_movies.itertuples():
    if actor1 in actor_dict:
        actor_dict[actor1] += count1
    else: 
        actor_dict[actor1] = count1

for index2, actor2, count2 in actor2_num_movies.itertuples():
    if actor2 in actor_dict:
        actor_dict[actor2] += count2
    else: 
        actor_dict[actor2] = count2
        
for index3, actor3, count3 in actor3_num_movies.itertuples():
    if actor3 in actor_dict:
        actor_dict[actor3] += count3
    else: 
        actor_dict[actor3] = count3  
        


In [19]:
#add actor count column to data frame
movies_after_1990['actor1_count'] = movies_after_1990['actor_1_name'].apply(lambda x: actor_dict.get(x))
movies_after_1990['actor2_count'] = movies_after_1990['actor_2_name'].apply(lambda x: actor_dict.get(x))
movies_after_1990['actor3_count'] = movies_after_1990['actor_3_name'].apply(lambda x: actor_dict.get(x))

4290
